In [1]:
## La version de python utilisée est 3.12.7

!pip install -r requirements.txt -q

In [2]:
import pandas as pd
import lxml as lxml
import io as io
import numpy as np

from script import modeling

In [3]:
transactions = pd.read_parquet("data/transactions_with_inondable_et_dpe.parquet",engine='pyarrow')

In [4]:
# Supposons que 'df' est déjà défini
# Étape 1 : Lister les noms des communes ayant au moins une "submersion marin"
communes_submersion = transactions.loc[transactions['libelle_type_inondation'] == "submersion marine", 'nom_commune'].unique()

# Étape 2 : Filtrer le DataFrame pour ne garder que ces communes
df = transactions[transactions['nom_commune'].isin(communes_submersion)]

In [5]:
# Étape 1 : Filtrer les communes n'ayant pas "submersion marine"
communes_non_submersion = transactions[~transactions['nom_commune'].isin(communes_submersion)][['nom_commune', 'Population']]

# Étape 2 : Supprimer les doublons si nécessaire (par commune) - si tu veux des lignes uniques
communes_non_submersion = communes_non_submersion.drop_duplicates(subset='nom_commune')

# Étape 3 : Trier par population décroissante
communes_non_submersion = communes_non_submersion.sort_values(by='Population', ascending=False).reset_index(drop=True)

# Afficher les 40 premières communes
display(communes_non_submersion.head(10))


,nom_commune,Population
0,Nantes,323204
1,Brest,139619
2,Perpignan,119656
3,Rouen,114083
4,Béziers,80341
5,Quimper,63642
6,Lorient,57846
7,Vannes,54420
8,Bayonne,52749
9,Arles,50415


In [6]:
df['terrain'] = (df['surface_terrain'] > 0).astype(int)

C:\Users\flori\AppData\Local\Temp\ipykernel_17544\129788132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['terrain'] = (df['surface_terrain'] > 0).astype(int)


In [7]:
# Décompte total des transactions par commune
decompte_communes = df['nom_commune'].value_counts()

# Décompte des transactions inondables par commune
decompte_inondables = df[df['zone_inondable'] == 1]['nom_commune'].value_counts()

# Convertir les deux décomptes en DataFrames
decompte_communes_df = decompte_communes.reset_index()
decompte_communes_df.columns = ['nom_commune', 'transactions']

decompte_inondables_df = decompte_inondables.reset_index()
decompte_inondables_df.columns = ['nom_commune', 'transactions_inondables']

# Fusionner les deux DataFrames sur le nom de la commune
decompte_final = pd.merge(decompte_communes_df, decompte_inondables_df, on='nom_commune', how='left')

# Remplacer les valeurs NaN par 0 (communes sans transactions inondables)
decompte_final['transactions_inondables'] = decompte_final['transactions_inondables'].fillna(0).astype(int)

# Fusionner avec le DataFrame initial pour conserver les autres colonnes de `df` (optionnel)
df = pd.merge(df, decompte_final, on='nom_commune', how='left')


In [8]:
df['dependance'] = df['dependance'].fillna(False).astype(int)

In [9]:
# Calcul de la distance pour chaque ligne du DataFrame
df['distance_centre_ville'] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_mairie'], row['longitude_mairie']
    ),
    axis=1
)

In [10]:
import ast

# Conversion des chaînes en listes de tuples
df["beach_coordinates"] = df["beach_coordinates"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

df["station"] = df["station"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [11]:
def nettoyer_coordinates(coords):
    if isinstance(coords, list):  # Si c'est déjà une liste, la garder
        return coords
    if isinstance(coords, float) or coords is None:  # Si c'est un NaN ou autre, retourner None
        return None
    return coords  # Par défaut, retourner la valeur initiale (si elle est correcte)

df["beach_coordinates"] = df["beach_coordinates"].apply(nettoyer_coordinates)
df["station"] = df["station"].apply(nettoyer_coordinates)

In [12]:
# Calcul de la distance minimale à la plage ou au port en fonction de la disponibilité des données
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["beach_coordinates"])
    if row["beach_coordinates"] else None,  # Si aucune coordonnée plage disponible, retourner None
    axis=1
)

# Si distance_min_beach est NaN, calculer la distance au port
df["distance_min_beach"] = df.apply(
    lambda row: modeling.distance_haversine(
        row['latitude'], row['longitude'],
        row['latitude_port'], row['longitude_port']
    ) if pd.isna(row["distance_min_beach"]) else row["distance_min_beach"],
    axis=1
)


In [13]:
df["distance_min_station"] = df.apply(
    lambda row: modeling.distance_minimale(row["latitude"], row["longitude"], row["station"])
    if row["station"] else None,  # Si aucune coordonnée disponible, retourner None
    axis=1
)

In [14]:
def convertir_en_numerique(df, colonnes):
    """
    Convertit les colonnes spécifiées du DataFrame en type numérique.
    
    Args:
        df (pd.DataFrame): Le DataFrame à modifier.
        colonnes (list): Liste des colonnes à convertir en numériques.
        
    Returns:
        pd.DataFrame: Le DataFrame avec les colonnes converties.
    """
    for col in colonnes:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [15]:
# Affichage des modalités de 'code_scenario'
modalites_code_scenario = df['code_scenario'].value_counts()
print(modalites_code_scenario)

code_scenario
02Moy    3312
04Fai    3282
01For    1269
03Mcc    1238
Name: count, dtype: int64


In [16]:
# Définir les bornes des tranches et les labels
bins = [0, 10000, 20000, np.inf]  # Définir les bornes des tranches
labels = ['0-10000','10000-20000','plus_20000']  # Labels des tranches

# Créer la variable catégorielle 'pop'
df['pop_cut'] = pd.cut(df['Population'], bins=bins, labels=labels, right=False)

# Créer des variables dummies pour les catégories de population
df_dummies_population = pd.get_dummies(df['pop_cut'], prefix='population',drop_first=True)
df_dummies_population = df_dummies_population.astype(int)

# Ajouter les dummies au DataFrame
df = pd.concat([df, df_dummies_population], axis=1)

In [17]:
# Fonction pour gérer la fusion des modalités et la création de dummies
def gerer_code_scenario(df, fusionner_modalites=True):
    # Si fusionner_modalites est True, fusionner les modalités '02Moy' et '03Mcc'
    if fusionner_modalites:
        df['code_scenario'] = df['code_scenario'].replace({
            '02Moy': '02Moy_03Mcc',
            '03Mcc': '02Moy_03Mcc'
        })
    
    # Création des dummies pour la colonne 'code_scenario'
    if 'code_scenario' in df.columns:  # Vérifie que la colonne existe
        df_dummies_scenario = pd.get_dummies(df['code_scenario'], prefix='scenario')
        df_dummies_scenario = df_dummies_scenario.astype(int)
    else:
        df_dummies_scenario = pd.DataFrame()  # Créer un DataFrame vide si aucune colonne
    
    # Ajouter les dummies au DataFrame original
    df = pd.concat([df, df_dummies_scenario], axis=1)
    
    return df

# Utilisation de la fonction
df = gerer_code_scenario(df, fusionner_modalites=True)  # Mettre False pour ne pas fusion


In [18]:
print(df['code_scenario'].value_counts(dropna=False))

code_scenario
None           49660
02Moy_03Mcc     4550
04Fai           3282
01For           1269
Name: count, dtype: int64


In [19]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_communes = pd.get_dummies(df['nom_commune'], prefix='commune')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_communes = df_dummies_communes.drop(columns=['commune_Toulon'])

# # Convertir les dummies en entiers (0/1)
df_dummies_communes = df_dummies_communes.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_communes], axis=1)

In [20]:
# Création des dummies pour la colonne 'departmentName'
df_dummies_department = pd.get_dummies(df['departmentName'], prefix='department')
# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df_dummies_department = df_dummies_department.drop(columns=['department_Alpes-Maritimes'])

# # Convertir les dummies en entiers (0/1)
df_dummies_department = df_dummies_department.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df = pd.concat([df, df_dummies_department], axis=1).drop(columns='departmentName')

In [21]:
# Appliquer le remplacement des modalités spécifiées
df['periode_construction_dpe'] = df['periode_construction_dpe'].replace({
    '2013-2021': 'après 2013',
    'après 2021': 'après 2013',
    '2001-2005': '2001-2012',
    '2006-2012': '2001-2012',
    '1983-1988': '1975-1988',
    '1978-1982': '1975-1988',
    '1975-1977': '1975-1988'
})

# Vérifier après remplacement
print(df['periode_construction_dpe'].unique())

# Création des dummies pour la colonne 'periode_construction_dpe'
df_dummies_construction = pd.get_dummies(df['periode_construction_dpe'], prefix='periode_construction_dpe')

# Supprimer la colonne inutile 'periode_construction_dpe_avant 1948' si elle existe
df_dummies_construction = df_dummies_construction.drop(columns=['periode_construction_dpe_avant 1948'], errors='ignore')

# Convertir les dummies en type entier
df_dummies_construction = df_dummies_construction.astype(int)

# Ajouter les dummies au DataFrame original
df = pd.concat([df, df_dummies_construction], axis=1)

[None '1948-1974' '1989-2000' '1975-1988' 'après 2013' 'avant 1948'
 '2001-2012']


In [22]:
# Création des dummies pour la colonne 'periode_construction_dpe'
df_dummies_dpe = pd.get_dummies(df['classe_bilan_dpe'], prefix='dpe')

df_dummies_dpe['dpe_E_F_G'] = df_dummies_dpe['dpe_G'] | df_dummies_dpe['dpe_F'] | df_dummies_dpe['dpe_E']

# Supprimer la colonne inutile 'periode_construction_dpe_avant 1948' si elle existe
df_dummies_dpe = df_dummies_dpe.drop(columns=['dpe_E_F_G'], errors='ignore')

# Convertir les dummies en type entier
df_dummies_dpe = df_dummies_dpe.astype(int)

# Ajouter les dummies au DataFrame original
df = pd.concat([df, df_dummies_dpe], axis=1)

# Supprimer les colonnes dpe_G et dpe_F originales
df = df.drop(columns=['dpe_G', 'dpe_F','dpe_E'])

In [23]:
# Conversion de la colonne 'dependance' en 0 et 1
df['dependance'] = df['dependance'].astype(int)

In [24]:

df2=df.dropna(subset=['periode_construction_dpe'])

# Vérifier le résultat final
print(df2.head())

   Unnamed: 0 id_mutation  numero_disposition  valeur_fonciere  \
1      140595  2023-77223                   1          95000.0   
2      130043  2023-72369                   1         371800.0   
4      130040  2023-72366                   1         261000.0   
5      130039  2023-72365                   1         215000.0   
6      122074  2023-64103                   1         650000.0   

   surface_reelle_bati  surface_terrain  nombre_locaux  dependance  \
1                 37.0              0.0              1           0   
2                 54.0              0.0              3           1   
4                 37.0              0.0              3           1   
5                 59.0              0.0              2           1   
6                130.0              0.0              3           1   

   nombre_pieces_principales date_mutation  ... department_Yonne  \
1                        2.0    2023-12-11  ...                0   
2                        2.0    2023-08-25  ..

In [25]:
count = df2[df2['nom_commune'] == "Nice"].shape[0]
print(count)

6766


In [26]:
# Filtrer les colonnes à conserver
colonnes_a_conserver = [col for col in df2.columns if not col.startswith('commune_')]

# Réassigner le DataFrame avec uniquement les colonnes restantes
df2 = df2[colonnes_a_conserver]

# Création des dummies pour la colonne 'departmentName'
df2_dummies_communes = pd.get_dummies(df2['nom_commune'], prefix='commune')

# # Si vous souhaitez supprimer une modalit/é particulière, vous pouvez utiliser .drop() :
df2_dummies_communes = df2_dummies_communes.drop(columns=['commune_Nice'])

# # Convertir les dummies en entiers (0/1)
df2_dummies_communes = df2_dummies_communes.astype(int)

# # Ajouter les dummies pour 'departmentName' au DataFrame original
df2 = pd.concat([df2, df2_dummies_communes], axis=1)

In [27]:
df2['debordement'] = (df2['libelle_type_inondation'] == "débordement de cours d'eau").astype(int)
df2['zone_inondable x debordement'] = df2['zone_inondable'] * df2['debordement']

In [28]:
# Transformation en log de certaines colonnes
variables_continues =    ["surface_reelle_bati",
                          "prix_m2",
                          "distance_min_beach",
                          "distance_min_station",
                          "distance_centre_ville"
                          ]

df = modeling.transformer_log(df, variables_continues)
df2 = modeling.transformer_log(df2, variables_continues)

df_app = df[df["type_local"]=="Appartement"]
df_app2 = df2[df2["type_local"]=="Appartement"]

# Colonnes explicatives, simplifiées à partir de variables_continues

colonnes_geographiques = [
    "log_distance_min_beach",
    "log_distance_min_station",
    "log_distance_centre_ville"
]

colonnes_explicatives = [
    "zone_inondable",
    "log_surface_reelle_bati",
    "nombre_pieces_principales",
    'terrain',
    'dependance'
    # "scenario_01For"
] + [
    col for col in df_app2.columns if col.startswith('population')
] + [
    col for col in df_app2.columns if col.startswith('dpe')
] + [
    col for col in df_app2.columns if col.startswith('periode_construction_dpe_')
]

colonnes_explicatives1 = colonnes_explicatives + [col for col in df_app2.columns if col.startswith('commune_')]

colonnes_explicatives2 = colonnes_explicatives1 + colonnes_geographiques

colonnes_explicatives3 = [
    col for col in colonnes_explicatives2 if col != "zone_inondable"
] + [
    col for col in df_app2.columns if col.startswith("scenario")
    ]
    
colonnes_explicatives4 = colonnes_explicatives2 + ['zone_inondable x debordement']


colonne_dependante = "log_prix_m2"

# Création du modèle avec dummies pour les départements
model_app1 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives1,
    colonne_dependante=colonne_dependante
)

model_app2 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives2,
    colonne_dependante=colonne_dependante
)

model_app3 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives3,
    colonne_dependante=colonne_dependante
)

model_app4 = modeling.construire_modele_regression(
    df_app2,
    colonnes_explicatives=colonnes_explicatives4,
    colonne_dependante=colonne_dependante
)

In [29]:
# Transformation en log de certaines colonnes
variables_continues =    ["surface_reelle_bati",
                          "prix_m2",
                          "distance_min_beach",
                          "distance_min_station",
                          "distance_centre_ville"
                          ]

df = modeling.transformer_log(df, variables_continues)
df2 = modeling.transformer_log(df2, variables_continues)

df_mai = df[df["type_local"]=="Maison"]
df_mai2 = df2[df2["type_local"]=="Maison"]

# Colonnes explicatives, simplifiées à partir de variables_continues

colonnes_geographiques = [
    "log_distance_min_beach",
    "log_distance_min_station",
    "log_distance_centre_ville"
]

colonnes_explicatives = [
    "zone_inondable",
    "log_surface_reelle_bati",
    "nombre_pieces_principales",
    "dependance",
    'terrain'
    # "scenario_01For"
] + [
    col for col in df_mai2.columns if col.startswith('population')
] + [
    col for col in df.columns if col.startswith('dpe')
] + [
    col for col in df_mai2.columns if col.startswith('periode_construction_dpe_')
]

colonnes_explicatives1 = colonnes_explicatives + [col for col in df_mai2.columns if col.startswith('commune_')]

colonnes_explicatives2 = colonnes_explicatives1 + colonnes_geographiques

colonnes_explicatives3 = [col for col in df_mai2.columns if col.startswith('scenario')] + [
    col for col in colonnes_explicatives3 if col != "zone_inondable"
    ] 

colonnes_explicatives4 = colonnes_explicatives2 + ['zone_inondable x debordement']

colonne_dependante = "log_prix_m2"

# Création du modèle avec dummies pour les départements

model_mai1 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives1,
    colonne_dependante=colonne_dependante
)

model_mai2 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives2,
    colonne_dependante=colonne_dependante
)

model_mai3 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives3,
    colonne_dependante=colonne_dependante
)

model_mai4 = modeling.construire_modele_regression(
    df_mai2,
    colonnes_explicatives=colonnes_explicatives4,
    colonne_dependante=colonne_dependante
)


In [30]:
# Extraction des résultats pour les modèles app
resultats_app1 = modeling.extraire_resultats_modele(model_app1, "A.1")
resultats_app2 = modeling.extraire_resultats_modele(model_app2, "A.2")
resultats_app3 = modeling.extraire_resultats_modele(model_app3, "A.3")
resultats_app4 = modeling.extraire_resultats_modele(model_app4, "A.4")

# Extraction des résultats pour les modèles mai
resultats_mai1 = modeling.extraire_resultats_modele(model_mai1, "M.1")
resultats_mai2 = modeling.extraire_resultats_modele(model_mai2, "M.2")
resultats_mai3 = modeling.extraire_resultats_modele(model_mai3, "M.3")
resultats_mai4 = modeling.extraire_resultats_modele(model_mai4, "M.4")

In [ ]:
# Retire les coefficients associés aux communes pour davantage de lisibilité
resultats_app1 = modeling.filtrer_variables(resultats_app1)
resultats_app2 = modeling.filtrer_variables(resultats_app2)
resultats_app3 = modeling.filtrer_variables(resultats_app3)
resultats_app4 = modeling.filtrer_variables(resultats_app4)

resultats_mai1 = modeling.filtrer_variables(resultats_mai1)
resultats_mai2 = modeling.filtrer_variables(resultats_mai2)
resultats_mai3 = modeling.filtrer_variables(resultats_mai3)
resultats_mai4 = modeling.filtrer_variables(resultats_mai4)

In [ ]:
# Effectuer les jointures sur la colonne 'variable' pour les appartements puis les maisons
tableau_app = resultats_app1.merge(resultats_app2, on="variable", how="outer")
tableau_app = tableau_app.merge(resultats_app3, on="variable", how="outer")
tableau_app = tableau_app.merge(resultats_app4, on="variable", how="outer")

tableau_mai = resultats_mai1.merge(resultats_mai2, on="variable", how="outer")
tableau_mai = tableau_mai.merge(resultats_mai3, on="variable", how="outer")
tableau_mai = tableau_mai.merge(resultats_mai4, on="variable", how="outer")

# Convertir les coefficients en float, puis arrondir à 4 chiffres après la virgule
coef_columns_app = [col for col in tableau_app.columns if "_coef" in col]
coef_columns_mai = [col for col in tableau_mai.columns if "_coef" in col]

# Convertir en float et arrondir
tableau_app[coef_columns_app] = tableau_app[coef_columns_app].apply(pd.to_numeric, errors='coerce')
tableau_mai[coef_columns_mai] = tableau_mai[coef_columns_mai].apply(pd.to_numeric, errors='coerce')

# Appliquer l'arrondi et les étoiles aux p-values
model_ids_app = [col.split("_")[0] for col in tableau_app.columns if "_pvalue" in col]
model_ids_mai = [col.split("_")[0] for col in tableau_mai.columns if "_pvalue" in col]

tableau_app = modeling.arrondir_pvalue_ajouter_etoiles(tableau_app, model_ids_app)
tableau_mai = modeling.arrondir_pvalue_ajouter_etoiles(tableau_mai, model_ids_mai)

In [ ]:
# Liste des variables dans l'ordre souhaité
ordre_variables_app = [
    'Observations',
    'R² ajusté',
    'const',
    'zone_inondable',
    'scenario_04Fai',
    'scenario_02Moy_03Mcc',
    'scenario_01For',
    'zone_inondable x debordement',
    'log_distance_centre_ville',
    'log_distance_min_beach',
    'log_distance_min_station',
    'log_surface_reelle_bati',
    'nombre_pieces_principales',
    'terrain',
    'dependance',
    'periode_construction_dpe_1948-1974',
    'periode_construction_dpe_1975-1988',
    'periode_construction_dpe_1989-2000',
    'periode_construction_dpe_2001-2012',
    'periode_construction_dpe_après 2013',
    'dpe_D',
    'dpe_C',
    'dpe_B',
    'dpe_A',
    'population_10000-20000',
    'population_20000-100000',
    'population_plus_100000'
]

ordre_variables_mai = [
    'Observations',
    'R² ajusté',
    'const',
    'zone_inondable',
    'scenario_04Fai',
    'scenario_02Moy_03Mcc',
    'scenario_01For',
    'zone_inondable x debordement',
    'log_distance_centre_ville',
    'log_distance_min_beach',
    'log_distance_min_station',
    'log_surface_reelle_bati',
    'nombre_pieces_principales',
    'terrain',
    'dependance',
    'periode_construction_dpe_1948-1974',
    'periode_construction_dpe_1975-1988',
    'periode_construction_dpe_1989-2000',
    'periode_construction_dpe_2001-2012',
    'periode_construction_dpe_après 2013',
    'dpe_D',
    'dpe_C',
    'dpe_B',
    'dpe_A',
    'population_10000-20000',
    'population_plus_20000'
]

# Réorganiser les lignes pour les deux tableaux (app et mai)
tableau_app = modeling.reordonner_lignes(tableau_app, ordre_variables_app)
tableau_mai = modeling.reordonner_lignes(tableau_mai, ordre_variables_mai)

# Appliquer le renommage des colonnes pour chaque tableau
tableau_app = modeling.renommer_coef_colonnes(tableau_app, "App")
tableau_mai = modeling.renommer_coef_colonnes(tableau_mai, "Maison")

In [31]:
tableau_app.fillna("").style.hide(axis="index")

variable,(App - M1),,(App - M2),,(App - M3),,(App - M4),
Observations,34676.000000,,30488.000000,,30488.000000,,30488.000000,
R² ajusté,0.288000,,0.282300,,0.282200,,0.285400,
const,8.829963,0.0***,8.656647,0.0***,8.656786,0.0***,8.621764,0.0***
zone_inondable,-0.025475,0.002***,-0.054104,0.0***,,,0.056766,0.0***
scenario_04Fai,,,,,-0.051235,0.0***,,
scenario_02Moy_03Mcc,,,,,-0.056375,0.0***,,
scenario_01For,,,,,-0.054033,0.008***,,
zone_inondable x debordement,,,,,,,-0.194959,0.0***
log_distance_centre_ville,,,0.051352,0.0***,0.051345,0.0***,0.052740,0.0***
log_distance_min_beach,,,-0.089440,0.0***,-0.089506,0.0***,-0.083502,0.0***


In [32]:
tableau_mai.fillna("").style.hide(axis="index")

variable,(Maison - M1),,(Maison - M2),,(Maison - M3),,(Maison - M4),
Observations,11420.000000,,8364.000000,,8364.000000,,8364.000000,
R² ajusté,0.523000,,0.538600,,0.538600,,0.539000,
const,9.046092,0.0***,8.881135,0.0***,8.875727,0.0***,8.875105,0.0***
zone_inondable,0.015164,0.257,-0.005042,0.76,,,0.018432,0.315
scenario_04Fai,,,,,-0.021819,0.138,,
scenario_04Fai,,,,,-0.021819,0.138,,
scenario_02Moy_03Mcc,,,,,0.000888,0.931,,
scenario_02Moy_03Mcc,,,,,0.000888,0.931,,
scenario_01For,,,,,0.020935,0.292,,
scenario_01For,,,,,0.020935,0.292,,
